In [28]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium  #needed for interactive map
from folium.plugins import HeatMap
import time
import datetime
from shapely.geometry import Polygon, Point
%matplotlib inline
sns.set()

In [29]:
bogota_shp = gpd.read_file("/Users/SebastianGuzman/Desktop/poligonos-localidades/poligonos-localidades.shp")
bogota_shp.head(2)

,Nombre_de_l,Acto_admini,Area_de_la_,Identificad,geometry
0,SANTA FE,Acuerdo 117 de 2003,"45170645,863322",03,"POLYGON ((-73.99446 4.61425, -73.99446 4.61425..."
1,BARRIOS UNIDOS,Acuerdo 8 de 1977,"11903448,204872",12,"POLYGON ((-74.05725 4.68684, -74.06249 4.65594..."


In [84]:
pd.set_option("display.max_columns", None)
tabla2 = pd.read_csv("tabla1.csv")
tabla2.head(2)

,ID,AGENTE,AUTOMOVILASIGNADO_ID,CADUCIDADASIGNACIONSERVICIO,CALIFICACIONCONDUCTORASIGNADO,CALIFICACION_USUARIO,CONDUCTORASIGNADO_ID,CONDUCTORESPOSTULADOS,COSTO,DELTACADUCIDADASIGNACION,DIRECCIONCOMPLEMENTARIA,DIRECCIONDESTINO,DIRECCIONPRINCIPAL,ESTADO,FECHA,FECHALEGIBLE,HORALANZAMIENTO,IDCIUDAD,IDLLAMADA,IDSERVICIO,INDICACIONES,LATITUD,LATITUDDESTINO,LONGITUD,LONGITUDDESTINO,MEDIOPAGO,NUMEROVALE,ORIGEN,PLACAAUTOMOVILASIGNADO,PROMEDIOCONDUCTORASIGNADO,PROPINA,TAMANOVEHICULO,TIPOASIGNACION,TIPOSERVICIO,USAPLANILLA,USUARIO,USUARIOAPLICACION_ID,USUARIOTELEFONICO_ID,IDCONDUCTORASIGNADO,DESCPAGO,BARRIOORIGEN,NUMEROCARROS,ANTERIOR,RUTA,CONDUCTOR_CERCANO,RECORRIDO_METROS,DISTANCIA,RECORRIDO_DISTANCIA,CALC_TARIFA,PRELIQUIDACION,TIEMPO,NUMERO_PASAJEROS,COSTO_INTERRUMPIDO
0,48167681,NaN,135018.0,90,NaN,NaN,153581868757140889910,...,11400.0,32,ATANASIO GIRARDOT,KR 50 # CL 6A (CALI),cl 38 # 13 39,CUMPLIDO,1617235199382,2021-04-01 00:00:00,NaN,76001,Sip/Cali_4444444,NaN,NaN,3.446157,3.414375,-76.507047,-76.545131,EFECTIVO,NaN,IVR,VCO258,5.0,NaN,NORMAL,TIEMPO_DISTANCIA,NORMAL,0,4333529,NaN,NaN,153581868757140889910,Pago definido/preliquidado por 11400,ATANASIO GIRARDOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48167682,NaN,106826.0,90,NaN,NaN,115066909299399545654,...,7500.0,60,CARLINA,DG 40A # AK 7 (BOGOTA),cl. 141 7b - 50,CUMPLIDO,1617235200439,2021-04-01 00:00:01,NaN,11001,17866503,NaN,PRPL,4.717940,4.625834,-74.030443,-74.066097,EFECTIVO,NaN,IVR,VDI561,5.0,NaN,NORMAL,TIEMPO_DISTANCIA,NORMAL,0,1243344,NaN,1243344.0,115066909299399545654,Pago definido/preliquidado por 7500,CEDRITOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
tabla2_df=tabla1[[
    "ID", "COSTO", "DIRECCIONDESTINO", "DIRECCIONPRINCIPAL", "ESTADO", "FECHA", "IDCIUDAD", "LATITUD", "LATITUDDESTINO",
    "LONGITUD", "LONGITUDDESTINO", "MEDIOPAGO"
    ]]
tabla2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  1000 non-null   int64  
 1   COSTO               787 non-null    float64
 2   DIRECCIONDESTINO    815 non-null    object 
 3   DIRECCIONPRINCIPAL  1000 non-null   object 
 4   ESTADO              1000 non-null   object 
 5   FECHA               1000 non-null   int64  
 6   IDCIUDAD            1000 non-null   int64  
 7   LATITUD             1000 non-null   float64
 8   LATITUDDESTINO      818 non-null    float64
 9   LONGITUD            1000 non-null   float64
 10  LONGITUDDESTINO     818 non-null    float64
 11  MEDIOPAGO           1000 non-null   object 
dtypes: float64(5), int64(3), object(4)
memory usage: 93.9+ KB


In [32]:
points = [(tabla1_df["ID"].iloc[i],Point(tabla1_df[["LONGITUD", "LATITUD"]].iloc[i].values)) for i in range(0,1000)]
localidad = []
for point in points:
    for polygon in bogota_shp["geometry"]:
        if polygon.contains(point[1]):
            localidad.append((point[0], bogota_shp[bogota_shp["geometry"] == polygon]["Nombre_de_l"].values[0]))

localidad_id = pd.DataFrame(localidad, columns=["ID", "LOCALIDAD"])
tabla1_df = pd.merge(tabla1_df, localidad_id, left_on="ID", right_on="ID", how="left")

In [33]:
points_destino = [(tabla1_df["ID"].iloc[i], Point(tabla1_df[["LONGITUDDESTINO", "LATITUDDESTINO"]].iloc[i].values)) for i in range(0,1000)]

localidad_destino = []
for point in points_destino:
    for polygon in bogota_shp["geometry"]:
        if polygon.contains(point[1]):
            localidad_destino.append((point[0], bogota_shp[bogota_shp["geometry"] == polygon]["Nombre_de_l"].values[0]))

localidad_id_destino = pd.DataFrame(localidad_destino, columns=["ID", "LOCALIDADDESTINO"])
tabla1_df = pd.merge(tabla1_df, localidad_id_destino, left_on="ID", right_on="ID", how="left")

In [34]:
tabla1_df.head(2)

,ID,COSTO,DIRECCIONDESTINO,DIRECCIONPRINCIPAL,ESTADO,FECHA,IDCIUDAD,LATITUD,LATITUDDESTINO,LONGITUD,LONGITUDDESTINO,MEDIOPAGO,LOCALIDAD,LOCALIDADDESTINO
0,48167681,11400.0,KR 50 # CL 6A (CALI),cl 38 # 13 39,CUMPLIDO,1617235199382,76001,3.446157,3.414375,-76.507047,-76.545131,EFECTIVO,NaN,NaN
1,48167682,7500.0,DG 40A # AK 7 (BOGOTA),cl. 141 7b - 50,CUMPLIDO,1617235200439,11001,4.717940,4.625834,-74.030443,-74.066097,EFECTIVO,USAQUEN,SANTA FE


In [36]:
tabla1_df["LOCALIDAD"]=tabla1_df["LOCALIDAD"].replace("ANTONIO NARIÃ\x91O", "ANTONIO NARINO")
tabla1_df["LOCALIDADDESTINO"]=tabla1_df["LOCALIDADDESTINO"].replace("ANTONIO NARIÃ\x91O", "ANTONIO NARINO")

In [20]:
tabla1_df.to_csv("tabla1_mod.csv")

In [80]:
bog_coords = [4.655085, -74.101300] # lat, long
folium_map = folium.Map(location=bog_coords, zoom_start=13, tiles="OpenStreetMap")

my_zip = zip(tabla1_df['LATITUD'], tabla1_df['LONGITUD'])
list_of_my_zip = list(my_zip)
for i in range(len(list_of_my_zip)):
    marker = folium.CircleMarker(location=[list_of_my_zip[i][0], list_of_my_zip[i][1]], radius = 2, color = "blue", fill = True)
    marker.add_to(folium_map)

location_destino=tabla1_df[["LATITUDDESTINO", "LONGITUDDESTINO"]].dropna()
list_of_my_zip_destino=list(zip(location_destino['LATITUDDESTINO'], location_destino['LONGITUDDESTINO']))
for i in range(len(list_of_my_zip_destino)):
    marker_destino = folium.CircleMarker(location=[list_of_my_zip_destino[i][0], list_of_my_zip_destino[i][1]], radius = 2, color = "red", fill = True)
    marker_destino.add_to(folium_map)

folium_map